In [70]:
# # !pip install -r requirement.txt
# import os

# # def list_distinct_file_names(directory):

# #     file_names = []
# #     distinct_filenames = set()
# #     for file_name in os.listdir(directory):
# #         if file_name.endswith('.jpg'):
# #             file_name_without_number = file_name[:-4]
# #             if file_name_without_number not in file_names:
# #                 file_names.append(file_name_without_number)
# #     for file in file_names:
# #         for c in file:
# #             if file[-1] != '_':
# #                 file = file.replace(file[-1], '')
# #         file = file[:-1]
# #         distinct_filenames.add(file)
        
# #     return list(distinct_filenames)


# # directory = 'images'
# # distinct_file_names = sorted(list_distinct_file_names(directory))
    
# # for file_name in distinct_file_names:
# #     directory_path = os.path.join(directory, file_name)
# #     os.makedirs(directory_path, exist_ok=True)
# #     print(file_name)
# source_directory = 'images'

# files = os.listdir(source_directory)

# # Loop through the files and move them to corresponding directories
# for file in files:
#     if '_' in file:
#         # Extract the part of the filename before the first underscore
#         prefix = file.split('_')[0]
        
#         # Create a destination directory if it doesn't exist
#         destination_directory = os.path.join(source_directory, prefix)
#         os.makedirs(destination_directory, exist_ok=True)
        
#         # Move the file to the destination directory
#         source_file = os.path.join(source_directory, file)
#         destination_file = os.path.join(destination_directory, file)
#         shutil.move(source_file, destination_file)

# print("Files moved successfully.")

Files moved successfully.


In [1]:
import numpy as np
import pandas as pd
import os, cv2, random, time, shutil, csv
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
np.random.seed(42)
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.nasnet import NASNetLarge, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
import concurrent.futures
from functools import partial
import asyncio
from PIL import Image

In [2]:
labels_dataframe = pd.read_csv('labels.csv')
dog_breeds = sorted(list(set(labels_dataframe['breed'])))

In [3]:
class_to_num = dict(zip(dog_breeds, range(120)))

In [5]:
class_to_num

{'affenpinscher': 0,
 'afghan_hound': 1,
 'african_hunting_dog': 2,
 'airedale': 3,
 'american_staffordshire_terrier': 4,
 'appenzeller': 5,
 'australian_terrier': 6,
 'basenji': 7,
 'basset': 8,
 'beagle': 9,
 'bedlington_terrier': 10,
 'bernese_mountain_dog': 11,
 'black-and-tan_coonhound': 12,
 'blenheim_spaniel': 13,
 'bloodhound': 14,
 'bluetick': 15,
 'border_collie': 16,
 'border_terrier': 17,
 'borzoi': 18,
 'boston_bull': 19,
 'bouvier_des_flandres': 20,
 'boxer': 21,
 'brabancon_griffon': 22,
 'briard': 23,
 'brittany_spaniel': 24,
 'bull_mastiff': 25,
 'cairn': 26,
 'cardigan': 27,
 'chesapeake_bay_retriever': 28,
 'chihuahua': 29,
 'chow': 30,
 'clumber': 31,
 'cocker_spaniel': 32,
 'collie': 33,
 'curly-coated_retriever': 34,
 'dandie_dinmont': 35,
 'dhole': 36,
 'dingo': 37,
 'doberman': 38,
 'english_foxhound': 39,
 'english_setter': 40,
 'english_springer': 41,
 'entlebucher': 42,
 'eskimo_dog': 43,
 'flat-coated_retriever': 44,
 'french_bulldog': 45,
 'german_shepherd'

In [4]:
def get_features(model, data_preprocessor, input_size, data):
    if model == NASNetLarge:
        input_layer = Input(shape=(331,331,3))
        preprocessor = Lambda(data_preprocessor)(input_layer)

    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    feature_maps = feature_extractor.predict(data, batch_size=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [10]:
dnn = keras.models.Sequential([
    InputLayer((5632,)),
    Dropout(0.7),
    Dense(120, activation='softmax')
])

dnn.load_weights("dog_final.h5")

dnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

ValueError: Cannot assign value to variable ' dense_1/kernel:0': Shape mismatch.The variable shape (5632, 120), and the assigned value shape (9664, 120) are incompatible.

In [6]:
InceptionV3 = InceptionV3(weights='inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)
Xception = Xception(weights='xception_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)
NASNetLarge = NASNetLarge(weights=None, include_top=False, input_shape=(331,331,3))
NASNetLarge.load_weights('NASNetLarge.h5')
InceptionResNetV2 = InceptionResNetV2(weights='inception.h5', include_top=False)

inception_preprocessor = keras.applications.inception_v3.preprocess_input
xception_preprocessor = keras.applications.xception.preprocess_input
nasnet_preprocessor = keras.applications.nasnet.preprocess_input
inc_resnet_preprocessor = keras.applications.inception_resnet_v2.preprocess_input

In [9]:

test_data = np.array(Image.open('n02085620_1271.jpg'))
test_data = tf.expand_dims(test_data, 0)
test_data.shape

input_size = (224,224,3)

inception_preprocessor = keras.applications.inception_v3.preprocess_input
xception_preprocessor = keras.applications.xception.preprocess_input
nasnet_preprocessor = keras.applications.nasnet.preprocess_input
inc_resnet_preprocessor = keras.applications.inception_resnet_v2.preprocess_input

inception_features = get_features(InceptionV3, inception_preprocessor, input_size, test_data)
xception_features = get_features(Xception, xception_preprocessor, input_size, test_data)
# nasnet_features = get_features(NASNetLarge, nasnet_preprocessor, input_size, test_data)
inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, input_size, test_data)
# num_threads = 4
# with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
#     parameters = [(InceptionV3, inception_preprocessor, input_size, test_data), (Xception, xception_preprocessor, input_size, test_data), (NASNetLarge, nasnet_preprocessor, input_size, test_data), (InceptionResNetV2, inc_resnet_preprocessor, input_size, test_data)]
#     inception_features = executor.submit(get_features, parameters[0])
#     xception_features = executor.submit(get_features, parameters[1])
#     nasnet_features = executor.submit(get_features, parameters[2])
#     inc_resnet_features = executor.submit(get_features, parameters[3])

# parameters = [(InceptionV3, inception_preprocessor), (Xception, xception_preprocessor), (NASNetLarge, nasnet_preprocessor), (InceptionResNetV2, inc_resnet_preprocessor)]

# with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:

#     results = []

#     def get_features_for_model(model, preprocessor):
#         return get_features(model, preprocessor, input_size, test_data)

#     for model, preprocessor in parameters:
#         future = executor.submit(get_features_for_model, model, preprocessor)
#         results.append(future)

#     # Wait for all tasks to complete and retrieve the results
#     concurrent.futures.wait(results)

#     # Get the results (features) from the completed tasks
#     inception_features, xception_features, nasnet_features, inc_resnet_features = [
#         future.result() for future in results
#     ]

test_features = np.concatenate([inception_features,
                                 xception_features,
                                 inc_resnet_features],axis=-1)


y_pred = dnn.predict(test_features)

top5_indices = np.argsort(y_pred[0])[::-1][:5]

for idx in top5_indices:
    class_name = [key for key, value in class_to_num.items() if value == idx][0]
    probability = y_pred[0][idx]
    print(f"Class: {class_name} \t\tProbability: {probability: .2f}")
    break;

1/1 [==============================] - 3s 3s/step
Feature maps shape:  (1, 2048)
1/1 [==============================] - 2s 2s/step
Feature maps shape:  (1, 2048)
1/1 [==============================] - 7s 7s/step
Feature maps shape:  (1, 1536)


ValueError: in user code:

    File "c:\Users\frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "c:\Users\frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [7]:

NASNetLarge = NASNetLarge(weights=None, include_top=False, input_shape=(331,331,3))
NASNetLarge.load_weights('NASNetLarge.h5')

ValueError: Layer count mismatch when loading weights from file. Model expected 526 layers, found 532 saved layers.